In [42]:
from tensorflow.keras import backend as K  # type: ignore
import tensorflow as tf # type: ignore
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # delete alert message from Tensorflow
from PIL import Image # type: ignore
from tensorflow.keras.preprocessing.image import img_to_array  # type: ignore
import numpy as np # type: ignore

In [43]:
y_true_path = "./test_mask/full_blue.png"
y_pred_path = "./test_mask/full_blue.png"

In [44]:
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    
    inputs = K.flatten(y_pred)
    targets = K.flatten(y_true)

    targets = K.expand_dims(targets, axis=-1)
    inputs = K.expand_dims(inputs, axis=-1)

    intersection = K.sum(targets * inputs)
    return (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)


def dice_coeff_multiclass(y_pred, y_true):

    num_class_pred = len(np.unique(y_pred))
    num_class_true = len(np.unique(y_true))
    if num_class_pred != num_class_true:
        num_class = np.unique(y_true)
    else :
        num_class = np.unique(y_true)

    el_unique = []
    for el in num_class:
        el_unique.append(el)

    dice_mean = []

    for value in el_unique:
        binary_pred = np.copy(y_pred)
        binary_pred[y_pred == value] = 0
        binary_pred[y_pred != value] = 1

        binary_true = np.copy(y_true)
        binary_true[y_true == value] = 0
        binary_true[y_true != value] = 1

        dice_mean.append(dice_coef(binary_true, binary_pred).numpy())

    return 1 - np.mean(dice_mean)


In [45]:
def iou(y_true, y_pred, smooth=1e-6):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')

    intersection = K.sum(K.abs(y_true * y_pred))
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    return K.mean((intersection + smooth) / (union + smooth))


def iou_multiclass(y_pred, y_true):
    num_class_pred = len(np.unique(y_pred))
    num_class_true = len(np.unique(y_true))
    if num_class_pred != num_class_true:
        num_class = np.unique(y_true)
    else :
        num_class = np.unique(y_true)

    el_unique = []
    for el in num_class:
        el_unique.append(el)

    dice_mean = []

    for value in el_unique:
        binary_pred = np.copy(y_pred)
        binary_pred[y_pred == value] = 0
        binary_pred[y_pred != value] = 1

        binary_true = np.copy(y_true)
        binary_true[y_true == value] = 0
        binary_true[y_true != value] = 1

        dice_mean.append(iou(binary_true, binary_pred).numpy())

    return 1 - np.mean(dice_mean)


In [46]:
input_shape = (256, 256, 3)

def transform_img(y_path):
    y_value = img_to_array(Image.open(y_path).resize(input_shape[:2], Image.NEAREST))
    y_value = y_value.astype(np.uint8)
    for x in range(y_value.shape[0]):
            for y in range(y_value.shape[1]):
                if (y_value[x, y] == [250, 170, 30]).all():
                    y_value[x, y] = 0
                elif (y_value[x, y] == [0, 0, 142]).all():
                    y_value[x, y] = 1
                elif (y_value[x, y] == [102, 102, 156]).all():
                    y_value[x, y] = 2
                elif (y_value[x, y] == [220, 20, 60]).all():
                    y_value[x, y] = 3
                elif (y_value[x, y] == [153, 153, 153]).all():
                    y_value[x, y] = 4
                elif (y_value[x, y] == [244, 35, 232]).all():
                    y_value[x, y] = 5
                elif (y_value[x, y] == [70, 70, 70]).all():
                    y_value[x, y] = 6
                elif (y_value[x, y] == [70, 130, 180]).all():
                    y_value[x, y] = 7
    y_value = y_value[:, :, 0:1]
    return y_value

y_true = transform_img(y_true_path)
y_pred = transform_img(y_pred_path)

In [47]:
print("Dice coef: ", dice_coeff_multiclass(y_pred, y_true))
print("Iou: ", iou_multiclass(y_pred, y_true))

Dice coef:  0.0
Iou:  0.0
